In [6]:
import sys
!{sys.executable} -m pip install psycopg2-binary

import psycopg2 as pg

In [7]:
conn = pg.connect(dbname="sequra", user="docker", password="docker", host="postgresql_sequra", port=5432)

## Load Mock Data into the database

In [8]:
with open("sql-1-fixtures.sql", "r") as file:
    sql = file.read()

cursor = conn.cursor()

try:
    cursor.execute(sql)
    conn.commit()
except (Exception, pg.DatabaseError) as error:
    print("Error: %s" % error)
    conn.rollback()
    cursor.close()

cursor.close()

## Solution

In [9]:
sql = """

WITH credits AS (

  SELECT 
    distinct(cd.credit_id) AS credit_id,
    cd.ext_fin_id AS financer_id
  FROM
    sql_1.fact_credit_d cd
  WHERE
    cd.record_date = (select max(cd.record_date) FROM sql_1.fact_credit_d cd WHERE DATE_PART('y', cd.record_date) = '2022')

),
financer AS (

  SELECT
    ef.ext_fin_id AS financer_id  
  FROM
    sql_1.dim_ext_fin ef
  WHERE
    ef.name = 'Fund2'

), 
financer_credits AS (

  SELECT 
    c.credit_id AS credit_id
  FROM
    credits c
  INNER JOIN 
    financer f ON c.financer_id = f.financer_id

)

  SELECT 
    evd.credit_id,
    evd.economic_event_id,
    evd.record_date,
    evd.type,
    evd.amount
  FROM
    sql_1.fact_economic_events_d evd
  INNER JOIN
    financer_credits fc ON fc.credit_id = evd.credit_id
  WHERE 
   type = 'chargeback' AND
   DATE_PART('y', record_date) = '2022';

"""

cursor = conn.cursor('sql_1_cursor')

try:
    
    cursor.itersize = 5000
    cursor.execute(sql)

    while True:
        rows = cursor.fetchmany(size=5000)

        for row in rows:
            print(row)
        
        if not rows:
            break

except (Exception, pg.DatabaseError) as error:
    print("Error: %s" % error)
    cursor.close()

cursor.close()

(17, 3, datetime.date(2022, 1, 7), 'chargeback', 10)
(18, 5, datetime.date(2022, 1, 10), 'chargeback', 50)


In [10]:
conn.close()